In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import calendar
import json
import csv
import numpy as np
from selenium import webdriver
import time
import math

In this Notebook we will collect data for the project called: *Analyzing the competition for the Hungarian fuel market*. We will webscrape our data from the following website: https://holtankoljak.net/. 

Sometimes if you run the script the login window will change to:

Elfelejtett jelszó / jelszó visszaállítás
A holtankoljak.hu (.net) rendszere nem tartalmazza az egyedi jelszavak un. tiszta, titkosítatlan formáját és a nagyobb biztonság érdekében olyan titkosítási eljárást használunk amely nem visszaállítható eredeti formájába.

In this case you should re-run the script until the real login page appears. Run the while cycle until it reaches the bottom of the page.

### Grabbing all the addresses of the Gas Stations:

In [108]:
username_mine = 'your username'
password_mine = 'your password'

#Launch Chrome and navigate to login page
driver = webdriver.Chrome('/home/csaba/Downloads/chromedriver')
driver.get("https://www.holtankoljak.hu/belepes")

#Click on 'OK tovább pop-up':
driver.find_element('xpath','/html/body/div[1]/div/div/div/div[2]/div/button[2]').click()

time.sleep(3)

#Click on 'Login' button:
driver.find_element('xpath', '/html/body/div[2]/div[2]/div/div[1]/button[1]').click()

time.sleep(2)

# Fill in login credentials
username = driver.find_element("xpath", "/html/body/div[4]/div/div/div[2]/form/div[1]/input")
password = driver.find_element("xpath", "/html/body/div[4]/div/div/div[2]/form/div[2]/input")

username.send_keys(username_mine)
password.send_keys(password_mine)

#Click login button with filled login data
driver.find_element('xpath', '/html/body/div[4]/div/div/div[2]/form/input[2]').click()

time.sleep(2)

#Now that we are logged in it is time to scrape down the toggle bar which contains the post code and the address of the gas stations
driver.find_element('xpath', '/html/body/div[1]/div[2]/div/div[1]/form[2]/div[2]/a').click()
driver.find_element('xpath', '/html/body/section/div/div[1]/div/div[2]/div[2]/form/div[1]/div/button').click()

time.sleep(10)

list_of_gas_stations = []


#we are going to let the script run to grab all the potential gas stations
while True:
    
    time.sleep(10)

    tags_of_current_gas_stations = driver.find_elements('xpath', "/html/body/div[6]/div/div[2]/ul/li")

    for station in tags_of_current_gas_stations:
        list_of_gas_stations.append(station.get_attribute('innerText'))

    list_of_gas_stations = list(set(list_of_gas_stations))

    tags_of_current_gas_stations[-1].location_once_scrolled_into_view

/tmp/ipykernel_4112/1426652612.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/csaba/Downloads/chromedriver')


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.110)
Stacktrace:
#0 0x55be1f327463 <unknown>
#1 0x55be1f0eb8d8 <unknown>
#2 0x55be1f0c7a50 <unknown>
#3 0x55be1f14e84d <unknown>
#4 0x55be1f1628d9 <unknown>
#5 0x55be1f14a443 <unknown>
#6 0x55be1f11b3c5 <unknown>
#7 0x55be1f11c531 <unknown>
#8 0x55be1f379dce <unknown>
#9 0x55be1f37d192 <unknown>
#10 0x55be1f35e93e <unknown>
#11 0x55be1f37e103 <unknown>
#12 0x55be1f351d85 <unknown>
#13 0x55be1f39f0a8 <unknown>
#14 0x55be1f39f239 <unknown>
#15 0x55be1f3ba492 <unknown>
#16 0x7f1dadf71609 start_thread


In [109]:
list_of_gas_stations = [i.replace('\xa0', '') for i in list_of_gas_stations]

len(list_of_gas_stations)

1291

In [110]:
len(set(list_of_gas_stations))

1291

This shows that we have unique elements. Let's save this list to our disk.

In [111]:
dic = {'Address with post code (Gas Stations)': list_of_gas_stations}  
list_of_gas_stations = pd.DataFrame(dic) 
list_of_gas_stations.to_csv('List_of_Gas_Stations.csv', index = False)

### Generating links and a DataFrame

In this part we would like to get a dataframe with the following format:

(Post Code) | (Franchise (or private)) | (Address) | (Is it next to railroad (0: No, 1: Yes)) | (Link of Gas Station at holtankoljak.hu)

In [2]:
gas_stations = pd.read_csv('List_of_Gas_Stations.csv')

In [3]:
gas_stations = gas_stations.loc[gas_stations['Address with post code (Gas Stations)'] != 'válassz....']

gas_stations['Address'] = gas_stations['Address with post code (Gas Stations)'].str.split('(').str[0].str[5:].str.rstrip()
gas_stations['City'] = gas_stations['Address'].str.split(',').str[0]
gas_stations['Post Code'] = gas_stations['Address with post code (Gas Stations)'].str[:4]
gas_stations['Franchise (or private)'] = gas_stations['Address with post code (Gas Stations)'].str.split('(').str[-1].str[:-1]


highway_numbers = [0,1,2,3,4,5,6,7,8,9,15,19,25,30,31,32,34,35,43,44,49,51,60,70,76,80,81,83,85,86,87,90,100]
highways = ['M{}'.format(i) for i in highway_numbers]

gas_stations['Highway'] = gas_stations['Address'].str.contains('|'.join(highways)).astype(int)

gas_stations['City_helper_for_links'] = gas_stations['City']
gas_stations.loc[gas_stations['City'].str.contains('Budapest'), 'City_helper_for_links'] = 'Budapest'

gas_stations

,Address with post code (Gas Stations),Address,City,Post Code,Franchise (or private),Highway,City_helper_for_links
0,"9024 Győr, Tatai út (mol)","Győr, Tatai út",Győr,9024,mol,0,Győr
1,"1152 Budapest XV, Szentmihályi út 131. (omv)","Budapest XV, Szentmihályi út 131.",Budapest XV,1152,omv,0,Budapest
2,"2400 Dunaújváros, Aranyvölgyi út 6. TESCO park...","Dunaújváros, Aranyvölgyi út 6. TESCO parkoló",Dunaújváros,2400,OIL!,0,Dunaújváros
3,"2461 Tárnok, M7 autópálya 26 km ÉSZAK (lukoil)","Tárnok, M7 autópálya 26 km ÉSZAK",Tárnok,2461,lukoil,1,Tárnok
4,"5520 Szeghalom, Kinizsi u. 12-14. (shell)","Szeghalom, Kinizsi u. 12-14.",Szeghalom,5520,shell,0,Szeghalom
...,...,...,...,...,...,...,...
1286,"7100 Szekszárd, Csatári u. (mol)","Szekszárd, Csatári u.",Szekszárd,7100,mol,0,Szekszárd
1287,"6050 Lajosmizse, Dózsa György u. 255 (Mobil Pe...","Lajosmizse, Dózsa György u. 255",Lajosmizse,6050,Mobil Petrol,0,Lajosmizse
1288,"4090 Polgár, M3 autópálya csomópont (Mobil Pet...","Polgár, M3 autópálya csomópont",Polgár,4090,Mobil Petrol,1,Polgár
1289,"8600 Siófok, Wesselényi u. 106. (mol)","Siófok, Wesselényi u. 106.",Siófok,8600,mol,0,Siófok


Some examples for understanding how to generate links from addresses:

- https://holtankoljak.net/mol_gyor_tataiut : 9024 Győr, Tatai út (mol) 
- https://holtankoljak.net/omv_budapest_xv_szentmihalyiut131 : 1152 Budapest XV, Szentmihályi út 131. (omv)
- https://holtankoljak.net/oil_dunaujvaros_aranyvolgyiut6tescoparkolo : 2400 Dunaújváros, Aranyvölgyi út 6. TESCO parkoló
- https://holtankoljak.net/lukoil_tarnok_m7autopalya26kmeszak : 2461 Tárnok, M7 autópálya 26 km ÉSZAK (lukoil)
- https://holtankoljak.net/shell_szeghalom_kinizsiu12-14 : 5520 Szeghalom, Kinizsi u. 12-14. (shell)

For testing we should use the following two examples to check whether or not we grabbed the real link:

- https://holtankoljak.net/oil_dunaujvaros_aranyvolgyiut6tescoparkolo: this is the good one, although
- https://holtankoljak.net/oil!_dunaujvaros_aranyvolgyiut6tescoparkolo: we will generate this (this is an outlier)


In [4]:
translation_table = str.maketrans('áéíóöőúüűÁÉÍÓÖŐÚÜŰ', 'aeiooouuuaeiiooouu')

gas_stations['Link'] = 'https://holtankoljak.net/' + gas_stations['Franchise (or private)'].str.replace(' ', '_').str.translate(translation_table).astype(str) + '_' + gas_stations['City_helper_for_links'].str.translate(translation_table).str.replace(' ', '_').astype(str) + '_'  + gas_stations['Address'].str.split(',').str[1].str.translate(translation_table).str.replace(' ', '').str.replace('.', '').str.replace('/', '').str.replace('-','').astype(str)

/tmp/ipykernel_649672/1040366976.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gas_stations['Link'] = 'https://holtankoljak.net/' + gas_stations['Franchise (or private)'].str.replace(' ', '_').str.translate(translation_table).astype(str) + '_' + gas_stations['City_helper_for_links'].str.translate(translation_table).str.replace(' ', '_').astype(str) + '_'  + gas_stations['Address'].str.split(',').str[1].str.translate(translation_table).str.replace(' ', '').str.replace('.', '').str.replace('/', '').str.replace('-','').astype(str)


In [5]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')

translation_table = str.maketrans('áéíóöőúüűÁÉÍÓÖŐÚÜŰ', 'aeiooouuuaeiiooouu')

bad_links = []

def getting_bad_links(link, city):
    '''
    This function returns the bad links (ie. those who does not contain addresses)
    '''
    
    driver = webdriver.Chrome('/home/csaba/Downloads/chromedriver', options=chrome_options)
    
    driver.get(link)
    time.sleep(3)
    
    try:
        driver.find_element('xpath', '/html/body/section/div/div[2]/div/div/div[2]/div[1]/div[1]')
        return link
            
    except:
        
        if 'Budapest' in city:
            
            try:
                
                link = link.split('Budapest_')[0] + city.translate(translation_table).replace(' ', '_') + '_' + link.split('Budapest_')[1]
                driver.get(link)
                time.sleep(3)
                driver.find_element('xpath', '/html/body/section/div/div[2]/div/div/div[2]/div[1]/div[1]')
                return link
            
            except:
                
                bad_links.append(link)
                print(link)
                return link

        bad_links.append(link)
        print(link)
        return link

#gas_stations['Real Links'] = gas_stations.apply(lambda x: getting_bad_links(x['Link'], x['City']), axis=1)

I would not recommend running the previous function, since it is slow and it is going through all the bad links that we fix in the next cell.

In [6]:
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/OIL!_Dunaujvaros_Aranyvolgyiut6TESCOparkolo', 'https://holtankoljak.net/oil_dunaujvaros_aranyvolgyiut6tescoparkolo')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Szeghalom_Kinizsiu1214', 'https://holtankoljak.net/shell_szeghalom_kinizsiu12-14#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Miskolc_ojgyorifoter3', 'https://holtankoljak.net/mol_miskolc_ujgyorifoter3')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XIII_RobertKarolykrt6466', 'https://holtankoljak.net/omv_budapest_robertkarolykrt64-66')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Veszprem_KulsoKadartaiu2', 'https://holtankoljak.net/omv_veszprem_kulso-kadartaiu2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Sumeg_Somoskertu27', 'https://holtankoljak.net/metrical_kft__sumeg_somoskertu27')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Kecskemet_44esFout82419hrsz', 'https://holtankoljak.net/lukoil_kecskemet_44-esfout82419hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Apagy_41esfoutBekeut55', 'https://holtankoljak.net/lukoil_apagy_41-esfoutbekeut55')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XI_Retkozutca', 'https://holtankoljak.net/mol_budapest_retkozutcaxi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Dorog_BecsiutCsolnokiutsarokhrsz02798', 'https://holtankoljak.net/omv_dorog_becsiut-csolnokiutsarokhrsz02798')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Kecskemet_Bethlenkrt5761', 'https://holtankoljak.net/shell_kecskemet_bethlenkrt57-61')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Lukoil_Zsambek_M1autopalya', 'https://holtankoljak.net/lukoil_zsambek_m1autopalyadel')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Magan_kut_Tata_Agostyaniut78', 'https://holtankoljak.net/magan_kut_tata_agostyaniut78zoldutkft')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Mobil_Petrol_Szentes_Attilaut4', 'https://holtankoljak.net/mobil_petrol_szentes_attilaut4attilaut-ipartelepiutkorf')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Oranges_Oil_Company_izd_Malomut1', 'https://holtankoljak.net/oranges_oil_company_ozd_malomut1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Gyor_Dombu24', 'https://holtankoljak.net/shell_gyor_dombu2-4')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_XVIII_MetauCziffraGyorgyutca', 'https://holtankoljak.net/shell_budapest_xviii_cziffragyorgyutca')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Godollo_M3aseszak', 'https://holtankoljak.net/lukoil_godollo_m3-aseszak')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Poroszlo_33asFout', 'https://holtankoljak.net/lukoil_poroszlo_33-asfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Kurd_Hunyadiu22a', 'https://holtankoljak.net/sza_oil_kft__kurd_hunyadiu22a#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Janoshalma_Halasiut91', 'https://holtankoljak.net/mol_janoshalma_halasiut9140993hrsz#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XI_Peterhegyiut6668', 'https://holtankoljak.net/omv_budapest_xi_peterhegyiut66-68')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Lukoil_Zsambek_M1autopalya', 'https://holtankoljak.net/lukoil_zsambek_m1autopalyaeszak')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Duna_Petrol_Kisapostag_6osfout76km', 'https://holtankoljak.net/duna_petrol_kisapostag_6-osfout76km#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_ojfeherto_4szfklut', 'https://holtankoljak.net/mol_ujfeherto_4szfklut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Vertesszolos_Hataru13', 'https://holtankoljak.net/mol_vertesszolos_hataru1-3')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Zalakomar_8752eszakINAbenzikutM7', 'https://holtankoljak.net/mol_zalakomar_8752eszak-inabenzikutm7#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_XI_Budafokiut209215', 'https://holtankoljak.net/shell_budapest_xi_budafokiut209-215')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Kazincbarcika_26szfoutBC4esporta3928hrsz', 'https://holtankoljak.net/mol_kazincbarcika_26szfoutbc4-esporta3928hrsz#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_VIII_KonyvesKalmankrt76', 'https://holtankoljak.net/omv_budapest_viii_konyveskalmankrt76konyveskobanyaisarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Kiskunfelegyhaza_Izsakiut22', 'https://holtankoljak.net/katep_tuker_kft__kiskunfelegyhaza_izsakiut22')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Tata_Ringu1', 'https://holtankoljak.net/shell_tata_ringu1m1autopalyalehajto')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_X_MaglodiutJaszberenyiutsarok', 'https://holtankoljak.net/omv_budapest_x_maglodiut-jaszberenyiutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_VIII_Hosu9', 'https://holtankoljak.net/mol_budapest_hosu9viii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Lajosmizse_M5', 'https://holtankoljak.net/shell_lajosmizse_m5lajosmizsekelet#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/auchan_Budaors_Sportu24', 'https://holtankoljak.net/auchan_budaors_sportu2-4')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Magocs_Vasarter8968hrsz', 'https://holtankoljak.net/magan_magocs_vasarter8968hrszadyeu79b')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_IX_Napfenyu26', 'https://holtankoljak.net/mol_budapest_napfenyu26ix')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Tabol_Sagvar_Foutca2', 'https://holtankoljak.net/tabol_kft__sagvar_foutca2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Bekecs_Honvedut228', 'https://holtankoljak.net/tobor_bt__bekecs_honvedut228')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Hajmasker_8asfout40km', 'https://holtankoljak.net/mol_hajmasker_8-asfout40km')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Kecskemet_Jokaiu44', 'https://holtankoljak.net/kis_kft__kecskemet_jokaiu44#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XIII_Vaciut178', 'https://holtankoljak.net/mol_budapest_vaciut178xiii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Mobil_Petrol_Kozarmisleny_57eses578asfoutkorforgalmaban', 'https://holtankoljak.net/mobil_petrol_kozarmisleny_57-eses578-asfoutkorforgalmaban')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_IV_Vaciut8490', 'https://holtankoljak.net/mol_budapest_vaciut84-90')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_ullo_Pestiut', 'https://holtankoljak.net/lukoil_ullo_pestiut4szfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Kecskemet_Kiskunfelegyhaziut', 'https://holtankoljak.net/mol_kecskemet_kiskunfelegyhaziutszegediut#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Dombovar_Koztarsasagu27', 'https://holtankoljak.net/mol_dombovar_koztarsasagu27tuskevar#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Tabol_Siofok_VakBottyanutca34', 'https://holtankoljak.net/tabol_kft__siofok_vak-bottyanutca34')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Tata_Vertesszolosiut', 'https://holtankoljak.net/mol_tata_vertesszolosiutbaloldal')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Sumeg_84esfout80km', 'https://holtankoljak.net/shell_sumeg_84-esfout80km')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Hatvan_M3IKerekharasztipihenoeszak', 'https://holtankoljak.net/mol_hatvan_m3i-kerekharasztipiheno-eszak')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Budapest_XVIII_Lakatosut6163', 'https://holtankoljak.net/szilas_ker__kft__budapest_lakatosut61-63')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_budapest_nagykorosiutxix88', 'https://holtankoljak.net/mol_budapest_nagykorosiutxix88')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Mobil_Petrol_Nyul_KossuthLajosut', 'https://holtankoljak.net/mobil_petrol_nyul_kossuthlajosut82szfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/MOL_Partner+_Akaszto_Fou18953asutmellett', 'https://holtankoljak.net/mol_partner+_akaszto_fou18953-asutmellett')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Tabol_Tab_Vasutallomasutca1', 'https://holtankoljak.net/tabol_kft__tab_vasutallomasutca1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Jaszszentlaszlo_Felegyhaziut2', 'https://holtankoljak.net/p_oil_kft__jaszszentlaszlo_felegyhaziut2#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Conoil_Papa_Celliu65', 'https://holtankoljak.net/concordia_trans_kft__papa_celliu65#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XXIII_icsaiukimeno', 'https://holtankoljak.net/mol_budapest_ocsaiukimenoxxiii7')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_VIII_Kerepesiut57', 'https://holtankoljak.net/mol_budapest_kerepesiut5-7#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Sarkeresztes_0133', 'https://holtankoljak.net/mol_sarkeresztes_013381-esfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Szecseny_Varsanyiu4', 'https://holtankoljak.net/sum_agro_kft__szecseny_varsanyiu4')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/edo_Szarvas_013192hrsz', 'https://holtankoljak.net/edo_szarvas_013192hrsz44-esszamufout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Nagyrede_Hrsz:02373', 'https://holtankoljak.net/omv_nagyrede_hrsz02373')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Berettyoujfalu_4247foutkeresztezodesHonvedu2', 'https://holtankoljak.net/mol_berettyoujfalu_47szfklut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Nyiregyhaza_Tokajiut', 'https://holtankoljak.net/mol_nyiregyhaza_tokajiutrepuloter')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Dombrad_Tiszakanyariut', 'https://holtankoljak.net/mol_dombrad_tiszakanyariuthagohegy')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_III_SzentendreiutCzetzJusarok', 'https://holtankoljak.net/omv_budapest_iii_szentendreiut-czetzjusarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/edo_Szeged_Szoregiut3941', 'https://holtankoljak.net/edo_szeged_szoregiut39-41#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Gorbehaza_M3IIdel', 'https://holtankoljak.net/mol_gorbehaza_m3iidel1825-oskmkohrsz04376#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Balatonkeresztur_M7Balatonkereszturdel', 'https://holtankoljak.net/shell_balatonkeresztur_m7balatonkeresztur-del')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XI_Budaorsiut1', 'https://holtankoljak.net/omv_budapest_xi_budaorsiut112')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Balatonlelle_M7II', 'https://holtankoljak.net/mol_balatonlelle_m7iibejovo')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Csem_13asfout', 'https://holtankoljak.net/lukoil_csem_13-asfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Mosonmagyarovar_M1M15autopalyaII', 'https://holtankoljak.net/mol_mosonmagyarovar_m1-m15autopalyaiipf109')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Nyiregyhaza_Mezou2840', 'https://holtankoljak.net/shell_nyiregyhaza_mezou28-40')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Eco_Smart_Group_God_NemeskeriKissMiklosut108', 'https://holtankoljak.net/eco_smart_group_god_nemeskeri-kissmiklosut108')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Kecskemet_44esfoutKuruczter', 'https://holtankoljak.net/lukoil_kecskemet_44-esfoutkuruczter')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XVII_Pestiut5', 'https://holtankoljak.net/mol_budapest_pestiut5xvii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_XIV_Mogyorodiut2232', 'https://holtankoljak.net/lukoil_budapest_xiv_mogyorodiut22-32#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Hodmezovasarhely_Hodtou1719', 'https://holtankoljak.net/shell_hodmezovasarhely_hodtou17-19')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mobil-petrol_Gyal_M5GyalAlsonemedilehajto', 'https://holtankoljak.net/mobil-petrol_gyal_m5gyal-alsonemedilehajto')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Hunpetrol_inod_Rakocziut2', 'https://holtankoljak.net/hunpetrol_onod_rakocziut2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Pecs_6szfklut', 'https://holtankoljak.net/mol_pecs_6szfklutcserkut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Fertod_Gaboraronutca11', 'https://holtankoljak.net/tank_ol_kft__fertod_gaboraronutca11')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Balatonmariafurdo_7esfout164km0541hrszBalatonkeresztur', 'https://holtankoljak.net/lukoil_balatonmariafurdo_7-esfout164km0541hrszbalatonkeresztur')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Mosonmagyarovar_M1M15autopalyaI', 'https://holtankoljak.net/mol_mosonmagyarovar_m1-m15autopalyaipf1091')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XIII_PapKarolyu', 'https://holtankoljak.net/mol_budapest_papkarolyuxiii#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_izd_Vaskoz12', 'https://holtankoljak.net/shell_ozd_vaskoz1-2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Karacsond_M3', 'https://holtankoljak.net/omv_karacsond_m3karacsondeszak#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_XV_M312kmDelSzilasPiheno', 'https://holtankoljak.net/shell_budapest_xv_m312kmdel-szilaspiheno')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XI_PrielleKorneliau', 'https://holtankoljak.net/mol_budapest_priellekorneliauxi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Kaposszekcso_611esfout010327hrsz', 'https://holtankoljak.net/mol_kaposszekcso_611-esfout010327hrszpetofiu')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Conoil_benzinkut_Tolmacs_Hrsz:013322a2esszamufoutTolmacsiLeagazas', 'https://holtankoljak.net/conoil_benzinkut_tolmacs_hrsz013322a2-esszamufouttolmacsileagazas#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/oil_Gyor_BanAladaru3Adyvaros', 'https://holtankoljak.net/oil_gyor_banaladaru3-adyvaros#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mobil-petrol_Gyor_SzigethyATancsicsMusarok', 'https://holtankoljak.net/mobil-petrol_gyor_szigethya-tancsicsmusarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_I_Alkotasu2024', 'https://holtankoljak.net/shell_budapest_i_alkotasu20-24')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Algyo_Szeged47szfklut', 'https://holtankoljak.net/mol_algyo_szeged47szfklutalgyopf11')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Pilisvorosvar_10szfklut', 'https://holtankoljak.net/mol_pilisvorosvar_10szfkluthrsz014031')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_erd_Budafokiut', 'https://holtankoljak.net/lukoil_erd_budafokiut6szfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XIX_PuskasFu', 'https://holtankoljak.net/mol_budapest_puskasfuxix')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Galambok_7esfout196km0245hrsz', 'https://holtankoljak.net/lukoil_galambok_7-esfout196km0245hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Ercsi_M6II', 'https://holtankoljak.net/mol_ercsi_m6iibejovom6kulterulet30kmszelveny')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Cegled_Nagykorosiut441szfklot', 'https://holtankoljak.net/mol_cegled_nagykorosiut441szfklut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Magan_kut_ullo_KSpedKrt', 'https://holtankoljak.net/magan_kut_ullo_k-spedkrtviktoriaiparipark')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Paty_22eskmIIdel', 'https://holtankoljak.net/mol_paty_22-eskmiidelsasfeszek-to')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Garden-Petrol_Budapest_XVII_Pestiut237', 'https://holtankoljak.net/garden-petrol_budapest_xvii_pestiut237osszekotoutca4-6')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XVI_VeresPeterut105107', 'https://holtankoljak.net/mol_budapest_verespeterut105-107xvi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Csemo_020817A', 'https://holtankoljak.net/magan_csemo_020817abenzinkuthrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Miskolc_KandoKKissTsarokKilianltp', 'https://holtankoljak.net/omv_miskolc_kandok-kisstsarok-kilianltp')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Enese_85osfout4339hrsz', 'https://holtankoljak.net/lukoil_enese_85-osfout4339hrsz#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Mezokeresztes_M3Gelej', 'https://holtankoljak.net/shell_mezokeresztes_m3gelejeszak')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XI_Irinyiut3436', 'https://holtankoljak.net/omv_budapest_xi_irinyiut34-36')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Dunaujvaros_Dunaujvarosnyugat', 'https://holtankoljak.net/omv_dunaujvaros_dunaujvaros-nyugatm6autopalya70100kmszbal')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_VIII_Golgotater', 'https://holtankoljak.net/mol_budapest_golgotaterviii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Dunakeszi_2esfout', 'https://holtankoljak.net/shell_dunakeszi_2-esfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_izd_Brassoiut5', 'https://holtankoljak.net/omv_ozd_brassoiut586314hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Oroszlany_Kornyeiu35', 'https://holtankoljak.net/shell_oroszlany_kornyeiu3-5')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Varpalota_8szfklutI', 'https://holtankoljak.net/mol_varpalota_8szfklutijobb')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XXI_KossuthLu3', 'https://holtankoljak.net/mol_budapest_kossuthlu3xxi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Hatvan_M3IIKerekharasztipihenoeszak', 'https://holtankoljak.net/mol_hatvan_m3ii-kerekharasztipiheno-eszak')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_IX_KonyvesKalmankrt1820', 'https://holtankoljak.net/shell_budapest_ix_konyveskalmankrt18-20')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Szazhalombatta_IpariPark6osfout0674hrszadalekoltimportuzemanyagertekesites', 'https://holtankoljak.net/magan_szazhalombatta_iparipark6-osfout0674hrsz-adalekoltimportuzemanyagertekesites')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/avia_Porboly_Vadaszutca1Hrsz:97', 'https://holtankoljak.net/avia_porboly_vadaszutca1hrsz97')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_ibarok_M143kmjobb', 'https://holtankoljak.net/omv_obarok_m143kmjobb#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Salgotarjan_Budapestiu21szfklot57km', 'https://holtankoljak.net/mol_salgotarjan_budapestiu21szfklut57km#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/avia_Tolna_Hrsz07746osfoutFacankertileagazo', 'https://holtankoljak.net/avia_tolna_hrsz07746-osfoutfacankertileagazo')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Ercsi_M6I', 'https://holtankoljak.net/mol_ercsi_m6ikimenom6kulterulet30kmszelveny')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Kal_Kapolnaiut12', 'https://holtankoljak.net/sprint_olaj_2005_kft__kal_kapolnaiut12#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Kunhegyes_KossuthLajosu7478', 'https://holtankoljak.net/mol_kunhegyes_kossuthlajosu74-78')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XV_Bogancsu', 'https://holtankoljak.net/mol_budapest_bogancsuxv#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Szekesfehervar_Lovoldeut', 'https://holtankoljak.net/omv_szekesfehervar_lovoldeutbudaiut39')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Szantod_7esfout', 'https://holtankoljak.net/lukoil_szantod_7-esfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Godollo_M3asDel', 'https://holtankoljak.net/lukoil_godollo_m3-asdel#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Bataszek_M6autopalya166kmhrsz:120118bal', 'https://holtankoljak.net/omv_bataszek_m6autopalya166kmhrsz120118jobb')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_VIII_Orczyut4648', 'https://holtankoljak.net/lukoil_budapest_viii_orczyut46-48')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_XV_M312kmeszakSzilasPiheno', 'https://holtankoljak.net/shell_budapest_xv_m312kmeszak-szilaspiheno')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Porcsalma_Szabadsagter15', 'https://holtankoljak.net/grovi_kft__porcsalma_szabadsagter15#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Sajtoskal_84esfout83km', 'https://holtankoljak.net/mol_sajtoskal_84-esfout83km')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/NK_Benzinkut_erd_Zamoriut4', 'https://holtankoljak.net/nagy_kut_bt__erd_zamoriut4')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Siofok_M7autopalya94km', 'https://holtankoljak.net/mol_siofok_m7autopalya94kmeszaksostoipihenoi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Miskolc_PestiutI', 'https://holtankoljak.net/mol_miskolc_pestiutibemeno')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Oplus_benzinkut_Nyiregyhaza_Tiszavasvariut', 'https://holtankoljak.net/oplus_benzinkut_nyiregyhaza_tiszavasvariutzomboribokor')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Csolnok_Kulterulethrsz01823', 'https://holtankoljak.net/tir_sped_kft__csolnok_kulterulethrsz01823')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XVIII_ulloiu738', 'https://holtankoljak.net/mol_budapest_ulloiu738xviii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Balatonederics_71esFout069hrsz', 'https://holtankoljak.net/mol_balatonederics_71-esfout069hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Full_Energy_Sopron_Somfalviu16', 'https://holtankoljak.net/full_sopron_ker__es_szolg__kft__sopron_somfalviu16')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/AVIA_Tapiobicske_ojut1', 'https://holtankoljak.net/avia_tapiobicske_ujut1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol-partner_Hernad_Koztarsasagut45DOUTBENZINKoTBT', 'https://holtankoljak.net/mol-partner_hernad_koztarsasagut45doutbenzinkutbt')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Komlo_KossuthLu5456', 'https://holtankoljak.net/shell_komlo_kossuthlu54-56')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XV_Alkotmanyut1420', 'https://holtankoljak.net/omv_budapest_xv_alkotmanyut14-20')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Mobil_Petrol_Kaposvar_arpadut3335', 'https://holtankoljak.net/mobil_petrol_kaposvar_arpadut33-35')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Fair-Kut_Hungary_Szakmar_BajcsyZsilinszkyu51', 'https://holtankoljak.net/fair-kut_hungary_szakmar_bajcsy-zsilinszkyu51')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Varpalota_8szfklutII', 'https://holtankoljak.net/mol_varpalota_8szfklutiibal')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Szolnok_Thokolyu', 'https://holtankoljak.net/omv_szolnok_thokolyut91#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Hevizgyork_AdyEndreut71', 'https://holtankoljak.net/goher_kft__hevizgyork_adyendreut71')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XVIII_HaladasutcaRadayutsarok', 'https://holtankoljak.net/omv_budapest_xviii_haladasutca-radayutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Balatonkeresztur_M7Balatonkeresztureszak', 'https://holtankoljak.net/shell_balatonkeresztur_m7balatonkeresztur-eszak#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XVI_SzlovakutCsomoriutsarok', 'https://holtankoljak.net/omv_budapest_xvi_szlovakut-csomoriutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Labatlan_Rakocziut36', 'https://holtankoljak.net/silopetrol_kft__labatlan_rakocziut36')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XIV_Ronau170', 'https://holtankoljak.net/mol_budapest_ronau170xiv')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Veszprem_KulsoKadartaiut3', 'https://holtankoljak.net/shell_veszprem_kulso-kadartaiut3')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Bataszek_Budaiut114116', 'https://holtankoljak.net/mol_bataszek_budaiut114-116')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/oil_icsa_KissJanosut1', 'https://holtankoljak.net/oil_ocsa_kissjanosut1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Oranges_OIL_Company_Keszthely_CsapasutGyenesdiasikorforgalomnal', 'https://holtankoljak.net/oranges_oil_company_keszthely_csapasut-gyenesdiasikorforgalomnal')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XIV_Hungariakrt175177', 'https://holtankoljak.net/omv_budapest_xiv_hungariakrt175-177')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Kerekegyhaza_Faludulo2', 'https://holtankoljak.net/kereken_palya_kft__kerekegyhaza_faludulo2#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Jaszjakohalma_Fout130b', 'https://holtankoljak.net/mol_jaszjakohalma_fout130b31szfout#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/AVIA_Iszkaszentgyorgy_801esutmellett', 'https://holtankoljak.net/avia_iszkaszentgyorgy_801-esutmellett')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/avia_Enying_DozsaGyot119', 'https://holtankoljak.net/avia_enying_dozsagyut119')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XIV_Vaganyu21', 'https://holtankoljak.net/mol_budapest_vaganyu21xiv')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Karacsond_M3', 'https://holtankoljak.net/omv_karacsond_m3karacsondeszak')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_XXI_Csepel', 'https://holtankoljak.net/shell_budapest_xxi_csepelrakocziferencut182')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Hatvan_Csanyiut', 'https://holtankoljak.net/mol_hatvan_csanyiutsportu1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Bonyhad_DrKoltaLaszlou', 'https://holtankoljak.net/bayer_kft__bonyhad_drkoltalaszlou#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Szarvas_Szabadsagut5153', 'https://holtankoljak.net/mol_szarvas_szabadsagut51-53')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol-partner_Dusnok_Bercsenyiu50', 'https://holtankoljak.net/mol-partner_dusnok_bercsenyiu5051szfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Hunpetrol_Tiszaujvaros_63411hrsz35osfoutmellett', 'https://holtankoljak.net/hunpetrol_tiszaujvaros_63411hrsz35-osfoutmellett')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_XIV_Egressyut3135', 'https://holtankoljak.net/shell_budapest_xiv_egressyut31-35')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Szeged_Bajaiut24', 'https://holtankoljak.net/shell_szeged_bajaiut2-4')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Fehergyarmat_Martirokutja12', 'https://holtankoljak.net/grovi_kft__fehergyarmat_martirokutja12')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Babolna_Sped__Babolna_Rakocziut', 'https://holtankoljak.net/babolna_sped__babolna_rakocziut1071hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_IX_Gyaliut', 'https://holtankoljak.net/lukoil_budapest_ix_gyaliutm5ecseriut9')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Budapest_XXI_CsepelMuvekFokapu', 'https://holtankoljak.net/magan_budapest_xxi_csepelmuvekfokapuszinesfemu23')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Sarkad_Vasarter13', 'https://holtankoljak.net/mol_sarkad_vasarter1-3#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Dunafoldvar_6osut84', 'https://holtankoljak.net/lukoil_dunafoldvar_6-osut84')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_X_Gyomroiut5662', 'https://holtankoljak.net/lukoil_budapest_x_gyomroiut56-62')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budaors_Repuloter', 'https://holtankoljak.net/mol_budaors_repuloterkinizsiu40472hrsz#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Dunafoldvar_6osut', 'https://holtankoljak.net/mol_dunafoldvar_6-osut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Oroshaza_Elkerulout', 'https://holtankoljak.net/mol_oroshaza_elkeruloutvasarhelyiut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Piliscsaba_10esfout160213hrsz', 'https://holtankoljak.net/mol_piliscsaba_10-esfout160213hrsz24km')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Magan_kut_Dudar_AranyJanosutca43hrsz:01337', 'https://holtankoljak.net/magan_kut_dudar_aranyjanosutca43hrsz01337#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Mobil_Petrol_Hodmezovasarhely_Kopancs47esszamufout', 'https://holtankoljak.net/mobil_petrol_hodmezovasarhely_kopancs47-esszamufout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budakeszi_Temetout', 'https://holtankoljak.net/mol_budakeszi_temetouthrsz2405')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budaors_Kinizsiut13', 'https://holtankoljak.net/shell_budaors_kinizsiut1-3')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Tabol_Igal_SzentIstvanutca187', 'https://holtankoljak.net/tabol_kft__igal_szentistvanutca187')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Kecskemet_Szechenyikrt1', 'https://holtankoljak.net/jo_ko_petrol_kft__kecskemet_szechenyikrt1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Mobil_Petrol_Miskolc_GyorikapuVologdausarok', 'https://holtankoljak.net/mobil_petrol_miskolc_gyorikapu-vologdausarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Nagyhalasz_Ibranyiut22', 'https://holtankoljak.net/csernyik_pal_e_v__nagyhalasz_ibranyiut22')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Eger_SasutMatyaskiralyu2', 'https://holtankoljak.net/omv_eger_sasut-matyaskiralyu2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XVI_AranyJu106', 'https://holtankoljak.net/mol_budapest_aranyju106xvi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Papa_Gyoriut', 'https://holtankoljak.net/mol_papa_gyoriutkulterulet')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/benz&benz_kft_ojhartyan_DragonBenztoltoallomas', 'https://holtankoljak.net/benz&benz_kft_ujhartyan_dragonbenztoltoallomas')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Oranges_Oil_Company_Zalaegerszeg_Balatoniu57Zalapark', 'https://holtankoljak.net/oranges_oil_company_zalaegerszeg_balatoniu5-7zalapark')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Fule_M7Autopalya83km', 'https://holtankoljak.net/mol_fule_m7autopalya83kmeszaklepseny')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_IX_Gubacsiu27', 'https://holtankoljak.net/mol_budapest_gubacsiu27ix')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Vecses_Gyaliut2022', 'https://holtankoljak.net/omv_vecses_gyaliut20-22')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Bohonye_61eses68ascsomopont', 'https://holtankoljak.net/lukoil_bohonye_61-eses68-ascsomopont#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Torokbalint_M0autopalya6kmszelv', 'https://holtankoljak.net/mol_torokbalint_m0autopalya6kmszelvannahegyipihenodel')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Canada_Petrol_Kecskemet_Felsocsalanosu54', 'https://holtankoljak.net/canada_petrol_kft__kecskemet_felsocsalanosu54')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_IX_HatarutSoroksariutsarok', 'https://holtankoljak.net/omv_budapest_ix_hatarut-soroksariutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Sopron_HataratkeloI', 'https://holtankoljak.net/mol_sopron_hataratkeloikimeno#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Unioil_Hegyeshalom_Mikloshalmiut1', 'https://holtankoljak.net/hatar_kut_hungary_kft__hegyeshalom_mikloshalmiut1#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Conoil_Gyor_Gyoriut2AMenfocsanak', 'https://holtankoljak.net/concordia_trans_kft__gyor_gyoriut2amenfocsanak#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_X_ulloiut110112', 'https://holtankoljak.net/omv_budapest_x_ulloiut110-112')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Paty_22eskmeszak', 'https://holtankoljak.net/mol_paty_22-eskmeszaksasfeszek-topaty')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/avia_Baja_BajcsyZsilinszkyutca56', 'https://holtankoljak.net/avia_baja_bajcsy-zsilinszkyutca56#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Polgar_Hajduu16', 'https://holtankoljak.net/mol_polgar_hajduu16munkacsyut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Baja_DozsaGyorgyut126130', 'https://holtankoljak.net/shell_baja_dozsagyorgyut126-130')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XIV_Hungariakrt116118', 'https://holtankoljak.net/omv_budapest_xiv_hungariakrt116-118')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Pecs_ZsolnayVu4143', 'https://holtankoljak.net/shell_pecs_zsolnayvu41-43')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Magan_kut_Debrecen_Dioszegiutca3234', 'https://holtankoljak.net/magan_kut_debrecen_dioszegiutca32-34')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Batonyterenye_izdiut36a', 'https://holtankoljak.net/lukoil_batonyterenye_ozdiut36a#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Debrecen_MikepercsiutII', 'https://holtankoljak.net/mol_debrecen_mikepercsiutiibejovo')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Ajka_Bakonygyepes', 'https://holtankoljak.net/mol_ajka_bakonygyepes8szfklut#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Gyongyos_3szfõut', 'https://holtankoljak.net/lukoil_gyongyos_3szf')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XIV_Tengerszemu6167', 'https://holtankoljak.net/omv_budapest_xiv_tengerszemu61-67#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Val_M7', 'https://holtankoljak.net/omv_val_m7kajaszovalieszak#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Jaszapati_31esFout99km02806hrsz', 'https://holtankoljak.net/lukoil_jaszapati_31-esfout99km02806hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/avia_Budapest_XXI_IIRakocziFerencu195197', 'https://holtankoljak.net/avia_budapest_iirakocziferencu195-197')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Mezokeresztes_M3', 'https://holtankoljak.net/shell_mezokeresztes_m3gelejeszak')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/avia_Hercegszanto_Hrsz:12162', 'https://holtankoljak.net/avia_hercegszanto_hrsz12162#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Diosd_Tetenyiu2', 'https://holtankoljak.net/mol_diosd_tetenyiu2hrsz36661#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/OIL!_Szodliget_2esfout27', 'https://holtankoljak.net/oil_szodliget_2-esfout27')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Pecs_Budaivam1', 'https://holtankoljak.net/shell_pecs_budaivam16szfoutmellett')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Dunaujvaros_AranyvolgyiutMagyaru', 'https://holtankoljak.net/oil_dunaujvaros_aranyvolgyiut6tescoparkolo#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_X_Feheru3', 'https://holtankoljak.net/mol_budapest_feheru3x')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Zalakomar_8752delINAbenzikutM7', 'https://holtankoljak.net/mol_zalakomar_8752del-inabenzikutm7#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_XIV_Kerepesiut4246', 'https://holtankoljak.net/shell_budapest_xiv_kerepesiut42-46')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Magan_kut_Paks_DunakomlodSzabadsagu1482292hrsz', 'https://holtankoljak.net/magan_kut_paks_dunakomlod-szabadsagu1482292hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Tabol_Siofok_BajcsyZsilinszkyutca214', 'https://holtankoljak.net/tabol_kft__siofok_bajcsy-zsilinszkyutca214')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XXI_SzentIstvanut', 'https://holtankoljak.net/mol_budapest_szentistvanutxxi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Babolna_BabolnaConcopihenoeszak', 'https://holtankoljak.net/omv_babolna_babolna-concopihenoeszakm1autopalya93kmsz#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XXIII_Helsinkiu108112', 'https://holtankoljak.net/mol_budapest_helsinkiu108-112')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Debrecen_MikepercsiutI', 'https://holtankoljak.net/mol_debrecen_mikepercsiutikimeno')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Albertirsa_4szfout5152km', 'https://holtankoljak.net/omv_albertirsa_4szfout51-52km')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Torokszentmiklos_KossuthLajosut9294', 'https://holtankoljak.net/mol_torokszentmiklos_kossuthlajosut92-94')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Lajosmizse_M5', 'https://holtankoljak.net/shell_lajosmizse_m5lajosmizsekelet#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Nagycserkesz_VasvariPalutca1026', 'https://holtankoljak.net/agro_cserkesz_kft__nagycserkesz_vasvaripalutca1026')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/auchan_Solymar_SzentFlorianu24', 'https://holtankoljak.net/auchan_solymar_szentflorianu2-4')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Mor_Nemesutca', 'https://holtankoljak.net/mol_mor_nemesu24')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Seregelyes_Dinnyesiut29', 'https://holtankoljak.net/unio_sped_2004_kft__seregelyes_dinnyesiut29')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Fonyod_JozsefAttilau', 'https://holtankoljak.net/mol_fonyod_jozsefattilaubelatelep#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XIV_Hungariakrt8385', 'https://holtankoljak.net/mol_budapest_hungariakrt83-85xiv')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Letenye_BajcsyZsilinszkiEndreu38', 'https://holtankoljak.net/mol_letenye_bajcsy-zsilinszkiendreu38#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Szentgotthard_8asut', 'https://holtankoljak.net/mol_szentgotthard_8-asut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Dunaharaszti_Fout220510esfout', 'https://holtankoljak.net/mol_dunaharaszti_fout220510-esfout#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XI_Irinyiu45', 'https://holtankoljak.net/mol_budapest_irinyiu45xi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Heves_Egriut0385hrsz', 'https://holtankoljak.net/toserne_leho_agnes_e_v__heves_egriut0385hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Tatabanya_SagvariutKarolyiMihalyu2', 'https://holtankoljak.net/omv_tatabanya_sagvariut-karolyimihalyu2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Conoil_Kormend_Sportutca5', 'https://holtankoljak.net/concordia_trans_kft__kormend_sportutca5tesco')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Kalocsa_Batyaiut810', 'https://holtankoljak.net/shell_kalocsa_batyaiut8-10#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Balatonalmadi_Fuzfoiut5', 'https://holtankoljak.net/mol_balatonalmadi_fuzfoiut571-esut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Sagvar_65osfout', 'https://holtankoljak.net/mol_sagvar_65-osfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_izd_Brassoiut3', 'https://holtankoljak.net/shell_ozd_brassoiut3')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Balatonlelle_M7I', 'https://holtankoljak.net/mol_balatonlelle_m7ikimeno')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XVIII_ulloiut661', 'https://holtankoljak.net/mol_budapest_ulloiut661xviii#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Hegyfalu_8486osfoutakkeresztezodese', 'https://holtankoljak.net/omv_hegyfalu_8486-osfoutakkeresztezodese#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Debrecen_Segnerter', 'https://holtankoljak.net/mol_debrecen_segnerternyugatiu6')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Kapuvar_SzentKatalinut45', 'https://holtankoljak.net/tank_ol_kft__kapuvar_szentkatalinut45')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Mezokeresztes_Kulterulet02592hrsz', 'https://holtankoljak.net/ferpetrol_kft__mezokeresztes_kulterulet02592hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Dunakeszi_2esszfout21km', 'https://holtankoljak.net/mol_dunakeszi_2-esszfout21km')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Bekesszentandras_44esut68km', 'https://holtankoljak.net/lukoil_bekesszentandras_44-esut68km')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_erd_Budafokiutca24', 'https://holtankoljak.net/shell_erd_budafokiutca2-4#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Kaposvar_Furediut', 'https://holtankoljak.net/omv_kaposvar_furediut537422hrsz#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol-partner_icsa_ulloiutca41', 'https://holtankoljak.net/mol-partner_ocsa_ulloiutca41')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Pusztamonostor_32esFout', 'https://holtankoljak.net/lukoil_pusztamonostor_32-esfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Fule_M7Autopalya83km', 'https://holtankoljak.net/mol_fule_m7autopalya83kmeszaklepseny')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Bonyhad_KossuthLajosu79', 'https://holtankoljak.net/mol_bonyhad_kossuthlajosu79bonyhadelkerulout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Varpalota_Veszpremiut1608asfout', 'https://holtankoljak.net/mol_varpalota_veszpremiut1608-asfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Sarvar_FeketehidutDeakFerencut', 'https://holtankoljak.net/shell_sarvar_feketehidut-deakferencut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_ibarok_M143kmbal', 'https://holtankoljak.net/omv_obarok_m143kmbal#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Miskolc_PestiutII', 'https://holtankoljak.net/mol_miskolc_pestiutiikijovo#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Nagykanizsa_ElkeruloutGaraiut17', 'https://holtankoljak.net/mol_nagykanizsa_elkerulout-garaiut17#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_IV_Vaciut77Megyeriutsarok', 'https://holtankoljak.net/omv_budapest_iv_vaciut77-megyeriutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Vecses_M0I', 'https://holtankoljak.net/mol_vecses_m0ijobb-alacskaipiheno')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Bataszek_M6autopalya166kmhrsz:120118jobb', 'https://holtankoljak.net/omv_bataszek_m6autopalya166kmhrsz120118jobb')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XI_Nagyszolosu4749', 'https://holtankoljak.net/mol_budapest_nagyszolosu47-4945662hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Gyorujbarat_M1Autopalya119km', 'https://holtankoljak.net/mol_gyorujbarat_m1autopalya119kmarrabona#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Autohof_ibarok_ibarok_M1autopalya43kmbaloldal', 'https://holtankoljak.net/omv_obarok_m143kmbal')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Godollo_3szFoutIpariPark', 'https://holtankoljak.net/mol_godollo_3szfoutipariparkpattantyusabrahamkrt4')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Esztergom_Dorogiut', 'https://holtankoljak.net/mol_esztergom_dorogiut111szfklut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/tele-tank_Magyarszek_kulterulet01343hrsz', 'https://holtankoljak.net/tele-tank_magyarszek_kulterulet01343hrsz66-osutmellett')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Dunakeszi_2szfõut', 'https://holtankoljak.net/lukoil_dunakeszi_2szf#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Bekescsaba_Szarvasiut1618', 'https://holtankoljak.net/omv_bekescsaba_szarvasiut16-18')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Cegled_Malomtoszelu1323', 'https://holtankoljak.net/shell_cegled_malomtoszelu13-23#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Mohacs_ojvaros', 'https://holtankoljak.net/shell_mohacs_ujvaros57szfout3403103hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Gyor_Budaiut', 'https://holtankoljak.net/mol_gyor_budaiutmartiroku#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XII_Istenhegyiu55', 'https://holtankoljak.net/mol_budapest_istenhegyiu55xii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Val_M7', 'https://holtankoljak.net/omv_val_m7kajaszovalieszak')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Szentgotthard_8asut', 'https://holtankoljak.net/lukoil_szentgotthard_8-asutszentgotthard035551hrszrabafuzes#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mobil-petrol_Budapest_X_SibrikGyomroisarok', 'https://holtankoljak.net/mobil-petrol_budapest_x_sibrik-gyomroisarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Vecses_M0II', 'https://holtankoljak.net/mol_vecses_m0iibal-alacskaipiheno')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_II_Tolgyfau13', 'https://holtankoljak.net/omv_budapest_ii_tolgyfau1-3')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/HMT_Petrol_Bekescsaba_BartokBelaut4244', 'https://holtankoljak.net/hmt_petrol_bekescsaba_bartokbelaut42-44#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XI_HunyadiJanosut2', 'https://holtankoljak.net/mol_budapest_hunyadijanosut2xi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Babolna_BabolnaConcopihenodel', 'https://holtankoljak.net/omv_babolna_babolna-concopihenodelm1autopalya39kmszbal')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mobil-petrol_Bag_M3autopalya39kmlehajto', 'https://holtankoljak.net/mobil-petrol_bag_m3autopalya39kmlehajtoiparipark2191')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_XIX_AdyEndreut2226', 'https://holtankoljak.net/mol_budapest_adyendreut22-26')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_XXII_Hajout', 'https://holtankoljak.net/omv_budapest_xxii_hajoutarteru5')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Monor_4szfklut', 'https://holtankoljak.net/mol_monor_4szfklutkatalincsarda#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Salgotarjan_Rakocziut7789', 'https://holtankoljak.net/mol_salgotarjan_rakocziut77-89')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_izd_Vasvarut', 'https://holtankoljak.net/mol_ozd_vasvarut25-osut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Oplus_benzinkut_Nyirtura_ojmajoriut10', 'https://holtankoljak.net/oplus_benzinkut_nyirtura_ujmajoriut10')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Gorbehaza_M3Ieszak', 'https://holtankoljak.net/mol_gorbehaza_m3ieszak1825-oskmkohrsz04378')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_kut_Solt_Foldvariut9799', 'https://holtankoljak.net/magan_kut_solt_foldvariut97-9952szfoutdelioldal5041hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Siofok_M7autopalya94km', 'https://holtankoljak.net/mol_siofok_m7autopalya94kmeszaksostoipihenoi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Szolnok_Hunyadiut24', 'https://holtankoljak.net/shell_szolnok_hunyadiut2-4')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_XXIII_icsaiuHRSZ1979695', 'https://holtankoljak.net/shell_budapest_xxiii_ocsaiuhrsz1979695')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Pacsa_75osFout', 'https://holtankoljak.net/lukoil_pacsa_75-osfout')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_VIII_Barossu5', 'https://holtankoljak.net/omv_budapest_viii_barossu5kalvinter')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Celldomolk_Wesselenyiu10', 'https://holtankoljak.net/cell_tank_kft___celldomolk_wesselenyiu10')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Dunaujvaros_Dunaujvaroskelet', 'https://holtankoljak.net/omv_dunaujvaros_dunaujvaros-keletm6autopalya70kmszjobb')

In [7]:
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_RobertKarolykrt6466', 'https://holtankoljak.net/omv_budapest_robertkarolykrt64-66')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Retkozutca', 'https://holtankoljak.net/mol_budapest_retkozutcaxi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_MetauCziffraGyorgyutca', 'https://holtankoljak.net/shell_budapest_xviii_cziffragyorgyutca')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Peterhegyiut6668', 'https://holtankoljak.net/omv_budapest_xi_peterhegyiut66-68')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Budafokiut209215', 'https://holtankoljak.net/shell_budapest_xi_budafokiut209-215')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_KonyvesKalmankrt76', 'https://holtankoljak.net/omv_budapest_viii_konyveskalmankrt76konyveskobanyaisarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_MaglodiutJaszberenyiutsarok', 'https://holtankoljak.net/omv_budapest_x_maglodiut-jaszberenyiutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Hosu9', 'https://holtankoljak.net/mol_budapest_hosu9viii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Napfenyu26', 'https://holtankoljak.net/mol_budapest_napfenyu26ix')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Vaciut178', 'https://holtankoljak.net/mol_budapest_vaciut178xiii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Vaciut8490', 'https://holtankoljak.net/mol_budapest_vaciut84-90')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Budapest_Lakatosut6163', 'https://holtankoljak.net/szilas_ker__kft__budapest_lakatosut61-63')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_icsaiukimeno', 'https://holtankoljak.net/mol_budapest_ocsaiukimenoxxiii7')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Kerepesiut57', 'https://holtankoljak.net/mol_budapest_kerepesiut5-7#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_SzentendreiutCzetzJusarok', 'https://holtankoljak.net/omv_budapest_iii_szentendreiut-czetzjusarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Budaorsiut1', 'https://holtankoljak.net/omv_budapest_xi_budaorsiut112')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Pestiut5', 'https://holtankoljak.net/mol_budapest_pestiut5xvii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_Mogyorodiut2232', 'https://holtankoljak.net/lukoil_budapest_xiv_mogyorodiut22-32#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_PapKarolyu', 'https://holtankoljak.net/mol_budapest_papkarolyuxiii#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_M312kmDelSzilasPiheno', 'https://holtankoljak.net/shell_budapest_xv_m312kmdel-szilaspiheno')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_PrielleKorneliau', 'https://holtankoljak.net/mol_budapest_priellekorneliauxi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Alkotasu2024', 'https://holtankoljak.net/shell_budapest_i_alkotasu20-24')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_PuskasFu', 'https://holtankoljak.net/mol_budapest_puskasfuxix')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Garden-Petrol_Budapest_Pestiut237', 'https://holtankoljak.net/garden-petrol_budapest_xvii_pestiut237osszekotoutca4-6')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_VeresPeterut105107', 'https://holtankoljak.net/mol_budapest_verespeterut105-107xvi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Irinyiut3436', 'https://holtankoljak.net/omv_budapest_xi_irinyiut34-36')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Golgotater', 'https://holtankoljak.net/mol_budapest_golgotaterviii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_KossuthLu3', 'https://holtankoljak.net/mol_budapest_kossuthlu3xxi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_KonyvesKalmankrt1820', 'https://holtankoljak.net/shell_budapest_ix_konyveskalmankrt18-20')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Salgotarjan_Budapestiu21szfklot57km', 'https://holtankoljak.net/mol_salgotarjan_budapestiu21szfklut57km#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Bogancsu', 'https://holtankoljak.net/mol_budapest_bogancsuxv#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_Orczyut4648', 'https://holtankoljak.net/lukoil_budapest_viii_orczyut46-48')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_M312kmeszakSzilasPiheno', 'https://holtankoljak.net/shell_budapest_xv_m312kmeszak-szilaspiheno')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_ulloiu738', 'https://holtankoljak.net/mol_budapest_ulloiu738xviii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Alkotmanyut1420', 'https://holtankoljak.net/omv_budapest_xv_alkotmanyut14-20')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_HaladasutcaRadayutsarok', 'https://holtankoljak.net/omv_budapest_xviii_haladasutca-radayutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_SzlovakutCsomoriutsarok', 'https://holtankoljak.net/omv_budapest_xvi_szlovakut-csomoriutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Ronau170', 'https://holtankoljak.net/mol_budapest_ronau170xiv')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Hungariakrt175177', 'https://holtankoljak.net/omv_budapest_xiv_hungariakrt175-177')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Vaganyu21', 'https://holtankoljak.net/mol_budapest_vaganyu21xiv')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Csepel', 'https://holtankoljak.net/shell_budapest_xxi_csepelrakocziferencut182')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Egressyut3135', 'https://holtankoljak.net/shell_budapest_xiv_egressyut31-35')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_Gyaliut', 'https://holtankoljak.net/lukoil_budapest_ix_gyaliutm5ecseriut9')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Budapest_CsepelMuvekFokapu', 'https://holtankoljak.net/magan_budapest_xxi_csepelmuvekfokapuszinesfemu23')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_Gyomroiut5662', 'https://holtankoljak.net/lukoil_budapest_x_gyomroiut56-62')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_AranyJu106', 'https://holtankoljak.net/mol_budapest_aranyju106xvi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Gubacsiu27', 'https://holtankoljak.net/mol_budapest_gubacsiu27ix')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_HatarutSoroksariutsarok', 'https://holtankoljak.net/omv_budapest_ix_hatarut-soroksariutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_ulloiut110112', 'https://holtankoljak.net/omv_budapest_x_ulloiut110-112')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Hungariakrt116118', 'https://holtankoljak.net/omv_budapest_xiv_hungariakrt116-118')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Tengerszemu6167', 'https://holtankoljak.net/omv_budapest_xiv_tengerszemu61-67#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/avia_Budapest_IIRakocziFerencu195197', 'https://holtankoljak.net/avia_budapest_iirakocziferencu195-197')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Feheru3', 'https://holtankoljak.net/mol_budapest_feheru3x')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Kerepesiut4246', 'https://holtankoljak.net/shell_budapest_xiv_kerepesiut42-46')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_SzentIstvanut', 'https://holtankoljak.net/mol_budapest_szentistvanutxxi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Helsinkiu108112', 'https://holtankoljak.net/mol_budapest_helsinkiu108-112')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Hungariakrt8385', 'https://holtankoljak.net/mol_budapest_hungariakrt83-85xiv')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Irinyiu45', 'https://holtankoljak.net/mol_budapest_irinyiu45xi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_ulloiut661', 'https://holtankoljak.net/mol_budapest_ulloiut661xviii#tartalom')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Vaciut77Megyeriutsarok', 'https://holtankoljak.net/omv_budapest_iv_vaciut77-megyeriutsarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Nagyszolosu4749', 'https://holtankoljak.net/mol_budapest_nagyszolosu47-4945662hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Istenhegyiu55', 'https://holtankoljak.net/mol_budapest_istenhegyiu55xii')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mobil-petrol_Budapest_SibrikGyomroisarok', 'https://holtankoljak.net/mobil-petrol_budapest_x_sibrik-gyomroisarok')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Tolgyfau13', 'https://holtankoljak.net/omv_budapest_ii_tolgyfau1-3')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_HunyadiJanosut2', 'https://holtankoljak.net/mol_budapest_hunyadijanosut2xi')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_AdyEndreut2226', 'https://holtankoljak.net/mol_budapest_adyendreut22-26')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Hajout', 'https://holtankoljak.net/omv_budapest_xxii_hajoutarteru5')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_icsaiuHRSZ1979695', 'https://holtankoljak.net/shell_budapest_xxiii_ocsaiuhrsz1979695')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Barossu5', 'https://holtankoljak.net/omv_budapest_viii_barossu5kalvinter')
#insert the last set of errors!
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Szentmihalyiut131', 'https://holtankoljak.net/omv_budapest_xv_szentmihalyiut131')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_SzilagyiErzsebetfasor51', 'https://holtankoljak.net/omv_budapest_ii_szilagyierzsebetfasor51')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Szentendreiut373', 'https://holtankoljak.net/shell_budapest_iii_szentendreiut373')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Kenu1', 'https://holtankoljak.net/shell_budapest_ix_kenu1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Budaorsiut112', 'https://holtankoljak.net/omv_budapest_xi_budaorsiut112')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Budapest_Rakospalotaihatarut14', 'https://holtankoljak.net/magan_budapest_xv_rakospalotaihatarut14')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_Esztenau1', 'https://holtankoljak.net/lukoil_budapest_xvii_esztenau1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Martirokutja281', 'https://holtankoljak.net/shell_budapest_xx_martirokutja281')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Nagykorosiut185', 'https://holtankoljak.net/omv_budapest_xx_nagykorosiut185')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Pestiut18', 'https://holtankoljak.net/omv_budapest_xvii_pestiut18')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_ulloiut233', 'https://holtankoljak.net/shell_budapest_xix_ulloiut233')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Szechenyiter1', 'https://holtankoljak.net/omv_budapest_xv_szechenyiter1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/mol_Budapest_Nagykorosiut', 'https://holtankoljak.net/mol_budapest_nagykorosiutxix88')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_DozsaGyorgyut120', 'https://holtankoljak.net/shell_budapest_vi_dozsagyorgyut120')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Budafokiut111', 'https://holtankoljak.net/omv_budapest_xi_budafokiut111')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Becsiut277A', 'https://holtankoljak.net/shell_budapest_iii_becsiut277a')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Kut_97_Budapest_Margitu114', 'https://holtankoljak.net/kut_97_budapest_xvi_margitu114')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Peterhegyiut54', 'https://holtankoljak.net/shell_budapest_xi_peterhegyiut54')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Balatoniut', 'https://holtankoljak.net/shell_budapest_xi_balatoniut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Budapest_Banyalegutca108', 'https://holtankoljak.net/magan_budapest_xxii_banyalegutca108')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Batthyanyut27', 'https://holtankoljak.net/shell_budapest_iii_batthyanyut27')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_HunyadiJut', 'https://holtankoljak.net/lukoil_budapest_xi_hunyadijut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_KacsohPongracut131', 'https://holtankoljak.net/omv_budapest_xiv_kacsohpongracut131')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Nagykorosiut349', 'https://holtankoljak.net/omv_budapest_xxiii_nagykorosiut349')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/auchan_Budapest_RakocziFerencut277Szigetszentmiklos', 'https://holtankoljak.net/auchan_budapest_xxi_rakocziferencut277szigetszentmiklos')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Fehervariut166', 'https://holtankoljak.net/omv_budapest_xi_fehervariut166')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Pozsonyiut1', 'https://holtankoljak.net/shell_budapest_iv_pozsonyiut1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Cinkotaiut51', 'https://holtankoljak.net/omv_budapest_xvii_cinkotaiut51')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/auchan_Budapest_HunyadiJanosu19SavoyaPark', 'https://holtankoljak.net/auchan_budapest_xi_hunyadijanosu19savoyapark')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_IIRakocziFerencut189', 'https://holtankoljak.net/omv_budapest_xxi_iirakocziferencut189')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Gyaliut30', 'https://holtankoljak.net/omv_budapest_ix_gyaliut30')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Telekiter', 'https://holtankoljak.net/omv_budapest_viii_telekiter')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Pestiut237c', 'https://holtankoljak.net/shell_budapest_xvii_pestiut237c')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_KorosiCsomaSandorut', 'https://holtankoljak.net/omv_budapest_x_korosicsomasandorut')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Fogarasiut56A', 'https://holtankoljak.net/omv_budapest_xiv_fogarasiut56a')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Bekeut58', 'https://holtankoljak.net/shell_budapest_xiii_bekeut58')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/avia_Budapest_Vagohidutca13', 'https://holtankoljak.net/avia_budapest_ix_vagohidutca13')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_SibrikMiklosut30', 'https://holtankoljak.net/shell_budapest_x_sibrikmiklosut30')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Fogarasiut183', 'https://holtankoljak.net/shell_budapest_xiv_fogarasiut183')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Tetenyiut2', 'https://holtankoljak.net/shell_budapest_xi_tetenyiut2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Meszarosu56A', 'https://holtankoljak.net/shell_budapest_i_meszarosu56a')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_Bokenyfoldiut108', 'https://holtankoljak.net/lukoil_budapest_xvi_bokenyfoldiut108')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Szentmihalyiut152', 'https://holtankoljak.net/shell_budapest_xv_szentmihalyiut152')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_BartokBelaut105', 'https://holtankoljak.net/lukoil_budapest_xi_bartokbelaut105')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Magan_Budapest_Maglodiut12B', 'https://holtankoljak.net/magan_budapest_x_maglodiut12b')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/auchan_Budapest_Bevasarlou2Soroksar', 'https://holtankoljak.net/auchan_budapest_xxiii_bevasarlou2soroksar')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/nkm-mobilitas_Budapest_Budaorsiut126', 'https://holtankoljak.net/nkm-mobilitas_budapest_xi_budaorsiut126')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Furediut63', 'https://holtankoljak.net/shell_budapest_xiv_furediut63')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Alacskaiut1', 'https://holtankoljak.net/omv_budapest_xviii_alacskaiut1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Mesteru89', 'https://holtankoljak.net/omv_budapest_ix_mesteru89')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Szabadfoldiut1', 'https://holtankoljak.net/omv_budapest_xvi_szabadfoldiut1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Kerepesiut61', 'https://holtankoljak.net/shell_budapest_x_kerepesiut61')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_SzilagyiEfasor125', 'https://holtankoljak.net/shell_budapest_ii_szilagyiefasor125')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Kobanyaiut56', 'https://holtankoljak.net/shell_budapest_x_kobanyaiut56')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Vaciut16', 'https://holtankoljak.net/omv_budapest_iv_vaciut16')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_ulloiut137', 'https://holtankoljak.net/omv_budapest_ix_ulloiut137')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Huvosvolgyiut118', 'https://holtankoljak.net/omv_budapest_ii_huvosvolgyiut118')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Vaciut196', 'https://holtankoljak.net/shell_budapest_xiii_vaciut196')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Mobil_Petrol_Budapest_Jaszberenyiut53', 'https://holtankoljak.net/mobil_petrol_budapest_x_jaszberenyiut53')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_KvassayJenout3', 'https://holtankoljak.net/lukoil_budapest_ix_kvassayjenout3')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Nagytetenyiut27', 'https://holtankoljak.net/shell_budapest_xxii_nagytetenyiut27')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Csaloganyu55', 'https://holtankoljak.net/shell_budapest_ii_csaloganyu55')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/nkm-mobilitas_Budapest_Kobanyaiut55', 'https://holtankoljak.net/nkm-mobilitas_budapest_x_kobanyaiut55')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/nkm-mobilitas_Budapest_igyallater1', 'https://holtankoljak.net/nkm-mobilitas_budapest_xiv_ogyallater1')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/magan_Budapest_Simongatutca2', 'https://holtankoljak.net/magan_budapest_xvi_simongatutca2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/NIT_Hungary_Budapest_Gyomroiut105', 'https://holtankoljak.net/nit_hungary_budapest_x_gyomroiut105')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/lukoil_Budapest_Pestiut2', 'https://holtankoljak.net/lukoil_budapest_xvii_pestiut2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Kerepesiut22', 'https://holtankoljak.net/omv_budapest_viii_kerepesiut22')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Vaciut95', 'https://holtankoljak.net/omv_budapest_xiii_vaciut95')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Kerepesiut73', 'https://holtankoljak.net/omv_budapest_x_kerepesiut73')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Gyomroiut156', 'https://holtankoljak.net/shell_budapest_xviii_gyomroiut156')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Harasztiut38b', 'https://holtankoljak.net/shell_budapest_xxiii_harasztiut38b')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/Mobil_Petrol_Budapest_Citroenutca2', 'https://holtankoljak.net/mobil_petrol_budapest_xix_citroenutca2')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_BalatoniutM1M775617hrsz', 'https://holtankoljak.net/omv_budapest_xi_balatoniutm1m775617hrsz')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Pestiut6', 'https://holtankoljak.net/shell_budapest_xvii_pestiut6')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Nepfurdou14', 'https://holtankoljak.net/shell_budapest_xiii_nepfurdou14')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Kiralyokutja178', 'https://holtankoljak.net/omv_budapest_iii_kiralyokutja178')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Budaorsiut118', 'https://holtankoljak.net/shell_budapest_xi_budaorsiut118')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/omv_Budapest_Harasztiut32A', 'https://holtankoljak.net/omv_budapest_xxiii_harasztiut32a')
gas_stations['Link'] = gas_stations['Link'].replace('https://holtankoljak.net/shell_Budapest_Halleru56', 'https://holtankoljak.net/shell_budapest_ix_halleru56')

In [8]:
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/eon_Szekesfehervar_Gazutca2']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/omv_Esztergom_Kertvaros']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/Magan_kut_Solt_Vasutallomasrakodo']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/NKM_Mobilitas_Kaposvar_Varosligetut']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/magan_Debrecen_Szechenyiut57']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/shell_Kazincbarcika_Joszerencsetut16']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/mol_budapest_metau-zadors']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/mol_Budapest_XIX_MetauZadors']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/STEINER-AUTOHOF_Bicske_M1autopalyamellett']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/magan_Fadd_AranyJanosu06811hrszJelenlegzarva!']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/nkm-mobilitas_Budapest_XIV_igyallater1']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/mobil_gaz_kft_Budapest_XXI_IIRakocziFerencut289295']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/magan_Pecs_Sportu9']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/mol_Budaors_METRO']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/Persped_Veszprem_Hazgyariu16']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/magan_Levelek_Rakocziutca50VoltTSZtelep']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/magan_Kalocsa_Batyaiu4']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/mobil_gaz_kft_Budapest_Bojtarutca78']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/avia_Budapest_Kereszturiut108a']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/Cheers_Cafe_Budapest_Repuloteriut6VasvariIpariPark']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/magan_hevizgyork_adyendreut71elado']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/magan_Tolna_6osutmellett130kmszelveny10105hrsz']
gas_stations = gas_stations.loc[gas_stations['Link'] != 'https://holtankoljak.net/mobil_gaz_kft_Budapest_IIRakocziFerencut289295']

In [9]:
gas_stations = gas_stations.drop(['City_helper_for_links'], axis = 1)
gas_stations

,Address with post code (Gas Stations),Address,City,Post Code,Franchise (or private),Highway,Link
0,"9024 Győr, Tatai út (mol)","Győr, Tatai út",Győr,9024,mol,0,https://holtankoljak.net/mol_Gyor_Tataiut
1,"1152 Budapest XV, Szentmihályi út 131. (omv)","Budapest XV, Szentmihályi út 131.",Budapest XV,1152,omv,0,https://holtankoljak.net/omv_budapest_xv_szent...
2,"2400 Dunaújváros, Aranyvölgyi út 6. TESCO park...","Dunaújváros, Aranyvölgyi út 6. TESCO parkoló",Dunaújváros,2400,OIL!,0,https://holtankoljak.net/oil_dunaujvaros_arany...
3,"2461 Tárnok, M7 autópálya 26 km ÉSZAK (lukoil)","Tárnok, M7 autópálya 26 km ÉSZAK",Tárnok,2461,lukoil,1,https://holtankoljak.net/lukoil_Tarnok_M7autop...
4,"5520 Szeghalom, Kinizsi u. 12-14. (shell)","Szeghalom, Kinizsi u. 12-14.",Szeghalom,5520,shell,0,https://holtankoljak.net/shell_szeghalom_kiniz...
...,...,...,...,...,...,...,...
1286,"7100 Szekszárd, Csatári u. (mol)","Szekszárd, Csatári u.",Szekszárd,7100,mol,0,https://holtankoljak.net/mol_Szekszard_Csatariu
1287,"6050 Lajosmizse, Dózsa György u. 255 (Mobil Pe...","Lajosmizse, Dózsa György u. 255",Lajosmizse,6050,Mobil Petrol,0,https://holtankoljak.net/Mobil_Petrol_Lajosmiz...
1288,"4090 Polgár, M3 autópálya csomópont (Mobil Pet...","Polgár, M3 autópálya csomópont",Polgár,4090,Mobil Petrol,1,https://holtankoljak.net/Mobil_Petrol_Polgar_M...
1289,"8600 Siófok, Wesselényi u. 106. (mol)","Siófok, Wesselényi u. 106.",Siófok,8600,mol,0,https://holtankoljak.net/mol_Siofok_Wesselenyi...


In [10]:
gas_stations.to_csv('gas_stations_with_link.csv', index=False)